In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

In [29]:
'''
Importing California SNG
'''

graph = src.graph.graph_from_json('Outputs/graph_full_red.json')

In [30]:
graph = nx.to_directed(graph)

deep_reload(src)

criteria = [
    lambda e: e['distance'] <= 550e3,
    lambda e: e['distance'] >= 10e3,
]

graph = src.graph.remove_edges(graph, criteria)

In [31]:
graph.is_directed(), graph.number_of_edges()

(True, 3613502)

In [32]:
'''
Next step is to define vehicle andsupply station objects.
These objects will be used to add supply costs to edges which begin at a supply station.
'''
deep_reload(src)

seed = 1468952
# seed = None
rng = np.random.default_rng(seed)

risk_attitude = (0, .2)
risk_attitude = (.8, 1)
rho = np.linspace(*risk_attitude, 100)

cases = 30

station_kw = {}

station_kw['place'] = {
    'cases': 1,
    'type': 'ac',
    'access': 'private',
    'price': .4 / 3.6e6,
    'setup_time': 0,
    'rng': rng,
}

station_kw['station'] = {
    'reliability': .5,
    'cases': cases,
    'type': 'dc',
    'access': 'public',
    'power': src.routing._network_power,
    'price': .5 / 3.6e6,
    'setup_time': 300,
    'rng': rng,
}

vehicle_kw = {
    'cases': 1,
    'linear_fraction': .8,
    'risk_attitude': (0, 1),
    # 'max_charge_start_soc': .8,
}

vehicle = src.routing.Vehicle(**vehicle_kw)

graph = src.routing.supply_costs(graph, vehicle, station_kw)

In [17]:
# graph._adj['station_10']

In [ ]:
deep_reload(src)

# vehicle_kw['max_charge_start_soc'] = .5
# vehicle = src.routing.Vehicle(**vehicle_kw)

# destinations = [k for k, v in graph._node.items() if v['type'] == 'place']
# origins = [destinations[0]]

node_to_idx = {k: idx for idx, k in enumerate(graph.nodes)}
idx_to_node = {idx: k for idx, k in enumerate(graph.nodes)}

places = [node_to_idx[k] for k, v in graph._node.items() if v['type'] == 'place']
stations = [node_to_idx[k] for k, v in graph._node.items() if v['type'] == 'station']

t0 = time.time()

costs, values, paths = src.floyd_warshall.floyd_warshall(
    graph, fields = ['routing_time'],
    origins = places,
    destinations = places,
    pivots = stations,
)

print(f'Executed in {time.time() - t0:.4f} seconds')

In [20]:
values[places[11]]

{0: {'routing_time': 0.0},
 1: {'routing_time': 0.0},
 2: {'routing_time': 0.0},
 3: {'routing_time': 0.0},
 4: {'routing_time': 0.0},
 5: {'routing_time': 0.0},
 6: {'routing_time': 0.0},
 7: {'routing_time': 0.0},
 8: {'routing_time': 0.0},
 9: {'routing_time': 0.0},
 10: {'routing_time': 0.0},
 11: {'routing_time': 0},
 12: {'routing_time': 0.0},
 13: {'routing_time': 0.0},
 14: {'routing_time': 0.0},
 15: {'routing_time': 0.0},
 16: {'routing_time': 0.0},
 17: {'routing_time': 0.0},
 18: {'routing_time': 0.0},
 19: {'routing_time': 0.0},
 20: {'routing_time': 0.0},
 21: {'routing_time': 0.0},
 22: {'routing_time': 0.0},
 23: {'routing_time': 0.0},
 24: {'routing_time': 0.0},
 25: {'routing_time': 0.0},
 26: {'routing_time': 0.0},
 27: {'routing_time': 0.0},
 28: {'routing_time': 0.0},
 29: {'routing_time': 0.0},
 30: {'routing_time': 0.0},
 31: {'routing_time': 0.0},
 32: {'routing_time': 0.0},
 33: {'routing_time': 0.0},
 34: {'routing_time': 0.0},
 35: {'routing_time': 0.0},
 36: